In [ ]:
#import 
import os, sys
sys.path.insert(1, os.path.abspath(".."))
import math
from sklearn.manifold import TSNE
import pickle
import torch
from torch import nn
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA, KernelPCA
#from autoencoder.plots import *
import os
from data_generation.utils.geom_utils import graph_distance
from settings.custom_settings import CustomSettings
from data_generation.generate_trajectory import Trajectory
from copy import deepcopy
from mpl_toolkits.mplot3d import axes3d
from training.train_ae import preprocessing
import pandas as pd
from matplotlib.path import Path
import matplotlib.patches as patches
from sklearn.preprocessing import StandardScaler

os.environ['KMP_DUPLICATE_LIB_OK']='True'


In [ ]:
EXP_NAME = "experiment5"  

In [ ]:
path = "../data_generation/generated_data/" + EXP_NAME
with open(path + "/maze.pkl", 'rb') as file:
    maze = pickle.load(file) #Maze is common to both
with open(path + "/trajectory.pkl", 'rb') as file:
    traj = pickle.load(file)
with open(path + "/placeCells.pkl", 'rb') as file:
    placeCells = pickle.load(file)

In [ ]:
 #build useful idx arrays
trajectory = np.array([traj.x_traj, traj.y_traj])
long_traj_maze_config = np.empty(trajectory.shape[1])
long_traj_maze_config[0:traj.traj_cut_idx[0]] = traj.corr_maze_config[0]
for i in range(len(traj.corr_maze_config)):
    cur_conf = traj.corr_maze_config[i]
    long_traj_maze_config[traj.traj_cut_idx[i]:traj.traj_cut_idx[i + 1]] = cur_conf
long_edge_pos = np.empty(trajectory.shape[1])
long_edge_pos[0:traj.traj_cut_idx[0]] = traj.corr_maze_config[0]
for i in range(len(traj.edge_position)):
    # nb_goals = (len(maze.goals[cur_conf]))
    long_edge_pos[traj.traj_cut_idx[i]:traj.traj_cut_idx[i + 1]] = traj.edge_position[i]

In [ ]:
FR = placeCells.firingRates

In [ ]:
edge = long_edge_pos
traj_2d= np.vstack((traj.x_traj, traj.y_traj)).T
maze_config = long_traj_maze_config


In [ ]:
with open('../rel_graph_distance/dgg_', 'rb') as file:
    dg = pickle.load(file)

# Useful Functions

In [ ]:
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])
#useful trajectory functions
def extract_umap_space_trajectory(traj, goal, maze_config, pc, model, scaler):
    idx = np.where((np.array(traj.edge_position) == goal) & (np.array(traj.corr_maze_config) == maze_config))[0][0]
    neural_activity = pc.firingRates[traj.traj_cut_idx[idx]:traj.traj_cut_idx[idx+1], :]
    neural_activity = scaler.transform(neural_activity)
    return model.transform(neural_activity)   

def extract_maze_trajectory(traj, goal, maze_config):
    idx = np.where((np.array(traj.edge_position) == goal) & (np.array(traj.corr_maze_config) == maze_config))[0][0]
    traj_x = traj.x_traj[traj.traj_cut_idx[idx]:traj.traj_cut_idx[idx+1]]
    traj_y = traj.y_traj[traj.traj_cut_idx[idx]:traj.traj_cut_idx[idx+1]]
    
    return traj_x, traj_y

#useful plotting functions
def ax_plot_traj(ax, traj_x_array, traj_y_array, maze, config, traj_col, maze_color="Greens", s_color="y"):
    #ax.title(title)
    col = sns.color_palette("colorblind")  
    ax.plot(traj_x_array, traj_y_array, color = traj_col)
    ax.plot(traj_x_array[0], traj_y_array[0], 'o', color = s_color)
    ax.plot(traj_x_array[-1], traj_y_array[-1], 'v', color = s_color)
    ax.set_xlim([0, maze.N])
    ax.set_ylim([0, maze.N])
    ax.grid()
    
        
        
def ax_plot_maze(ax, maze, config, plot_goals=[True, True, True], plot_home=True, maze_col = "Greys", ls = "-", hatch = None):
    col = sns.color_palette("colorblind")  
    patch = patches.PathPatch(maze.trialOctoMaze[config], facecolor='None', lw=3, linestyle = ls, hatch=hatch)
    ax.add_patch(patch)
    ax.imshow(maze.trialMazeFlags[config, ::-1], cmap=maze_col, extent=[0, maze.N, 0, maze.N], alpha=0.1)
    h = np.array(maze.nodeList[config]["A"]) + 0.5
    if plot_home:
        ax.scatter(h[0], h[1], s = 500, alpha = 1, marker = "P", color = col[0])
    for k in range(3): #goals
        gg = ["C", "D", "E"]
        if plot_goals[k]:
            g = np.array(maze.nodeList[config][gg[k]])+0.5
            ax.scatter(g[0], g[1], s = 500, color = col[k+2], marker = "8", alpha= 0.5) 

# UMAP

In [ ]:
import umap

In [ ]:
model = umap.UMAP(n_neighbors=150, min_dist = 0.001)

In [ ]:
scaler = StandardScaler().fit(FR)
data = scaler.transform(FR)

In [ ]:
df2 = pd.DataFrame(data)
#df2 = df2.sample(100000)
model.fit(df2)

In [ ]:
col = np.zeros((3, df2.index.shape[0]))
col[0, :] =  traj_2d[df2.index, 0]
col[2, :] =  traj_2d[df2.index, 1]
col = col/7

In [ ]:
embedding2 = model.transform(df2)

In [ ]:
df2.index

In [ ]:
plt.title("Graph hypothesis - UMAP")
s_i = np.arange(embedding2.shape[0])
np.random.shuffle(s_i)
plt.scatter(embedding2[s_i, 0], embedding2[s_i, 1], c=col[:, s_i].T, alpha = 0.5)

In [ ]:
plt.title("Graph hypothesis - UMAP")
plt.scatter(embedding2[:, 0], embedding2[:, 1], c=dg[df2.index])
plt.colorbar(label="Relative Graph distance from home", orientation="vertical")
plt.savefig("UMAP_graph.pdf")

In [ ]:
plt.title("Graph hypothesis - UMAP")

for i in range(3):
    idx = np.where(edge[df2.index] == i)
    plt.scatter(embedding2[idx, 0], embedding2[idx, 1], label = "goal %s"%i)

plt.legend()

## Trajectories

In [ ]:
maze_config = maze_config[df2.index]
edge = edge[df2.index]

In [ ]:
import seaborn as sns
col = sns.color_palette("colorblind")
col

In [ ]:
n_maze=3

#Plot trajectories in latent space
#EUCL
plt.figure()
fig, axs = plt.subplots(2, n_maze, figsize = (15, 10))

for m in range(n_maze):
    im = axs[0, m].scatter(embedding2[:, 0], embedding2[:, 1], color="grey")


for k in range(n_maze): #maze config
        #idx = np.where(maze_config_e == k)
    home = maze.nodeList[k]["A"]
    idx = np.where(maze_config == k)
    patch = patches.PathPatch(maze.trialOctoMaze[k], facecolor='None', lw=3)
    axs[1, k].add_patch(patch)
    
  
    
    for g in range(3): #goal edge
        traj_act_latent = extract_umap_space_trajectory(traj, goal=g, maze_config=k, pc=placeCells, model= model, scaler=scaler)
        
        #traj_act_latent= model.transform(traj)#traj_act_latent.detach().numpy())

        axs[0, k].scatter(traj_act_latent[:, 0], traj_act_latent[:, 1], color=col[g+2], label = "maze %s, goal %s"%(k, g))
        axs[0, k].plot(traj_act_latent[0, 0], traj_act_latent[0, 1], 'yo', markersize = 8)
        axs[0, k].plot(traj_act_latent[-1, 0], traj_act_latent[-1, 1], 'yx',  markersize = 8)        
        
        
        traj_x_array, traj_y_array = extract_maze_trajectory(traj, g, k)
        ax_plot_traj(axs[1, k], traj_x_array, traj_y_array, maze, k, traj_col = col[g+2])

In [ ]:
plt.figure()
fig, axs = plt.subplots(2, 1, figsize = (5, 10))


for g in range(3): #goal edge
    idx_g = np.where(edge == g)


    im = axs[0].scatter(embedding2[idx_g, 0], embedding2[idx_g, 1], alpha = 1, color = lighten_color(col[g+2]))

g=0    
for k in range(3): #maze config
    traj_act_latent = extract_umap_space_trajectory(traj, goal=g, maze_config=k, pc=placeCells, model= model, scaler=scaler)

    axs[0].plot(traj_act_latent[:, 0], traj_act_latent[:, 1], label = "maze %s, goal %s"%(k, g), c=col[g+2])
    axs[0].plot(traj_act_latent[0, 0], traj_act_latent[0, 1], 'yo', markersize = 8)
    axs[0].plot(traj_act_latent[-1, 0], traj_act_latent[-1, 1], 'yx',  markersize = 8)

    traj_x_array, traj_y_array = extract_maze_trajectory(traj, g, k)
    ax_plot_traj(axs[1], traj_x_array, traj_y_array, maze, k, traj_col = col[g+2])#, maze_color = colors[k])

    #patch = patches.PathPatch(maze.trialOctoMaze[k], facecolor='None', lw=3)
    #axs[1, g].add_patch(patch)
    ax_plot_maze(axs[1], maze, k)
plt.savefig("umap_graoh_traj.pdf")

# 3D UMAP

In [ ]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

In [ ]:
model3D = umap.UMAP(n_components=3, n_neighbors=200)

In [ ]:
df3 = pd.DataFrame(data)
#df3 = df3.sample(100000)
model3D.fit(df3)


In [ ]:

embedding = model3D.transform(df3)

In [ ]:
col = np.zeros((3, df3.index.shape[0]))
col[0, :] =  traj_2d[df3.index, 0]
col[2, :] =  traj_2d[df3.index, 1]

col = col/7*256


marker_dict = dict(size=3,
        color= [f'rgb({col[0, i]}, {col[1, i]}, {col[2, i]})' for i in range(col.shape[1])],  # set color to an array/list of desired values
        opacity=0.8)

fig = go.Figure()
fig.add_scatter3d(x=embedding[:, 0], y=embedding[:, 1], z=embedding[:, 2],mode='markers', marker=marker_dict) #, color = "Euclidean distance from home", title = "Euclidean Hyp: latent space" )

fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[embedding[:, 0].min(),embedding[:, 0].max()],),
                     yaxis = dict(nticks=4, range=[embedding[:, 1].min(),embedding[:, 1].max()],),
                     zaxis = dict(nticks=4, range=[embedding[:, 2].min(),embedding[:, 2].max()],),),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10), 
    title={
        'text': "Eucl",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


fig.show()


#legend plot
cp1 = np.linspace(0,1)
cp2 = np.linspace(0,1)
Cp1, Cp2 = np.meshgrid(cp1,cp2)
C0 = np.zeros_like(Cp1)# make RGB image, p1 to red channel, p2 to blue channel
Legend = np.dstack((Cp1, C0, Cp2))

# parameters range between 0 and 7
plt.figure(figsize=[1, 1])
plt.imshow(Legend, origin="lower", extent=[0,7,0,7])
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Euclidean Coordinates", fontsize=10)

In [ ]:
marker_dict = dict(size=3,
        color= dg[df3.index],  # set color to an array/list of desired values
        opacity=0.8)

fig = go.Figure()
fig.add_scatter3d(x=embedding[:, 0], y=embedding[:, 1], z=embedding[:, 2],mode='markers', marker=marker_dict) #, color = "Euclidean distance from home", title = "Euclidean Hyp: latent space" )

fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[embedding[:, 0].min(),embedding[:, 0].max()],),
                     yaxis = dict(nticks=4, range=[embedding[:, 1].min(),embedding[:, 1].max()],),
                     zaxis = dict(nticks=4, range=[embedding[:, 2].min(),embedding[:, 2].max()],),),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10), 
    title={
        'text': "Eucl",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


fig.show()

In [ ]:
marker_dict = dict(size=3,
        color= edge[df3.index],  # set color to an array/list of desired values
        opacity=0.8)

fig = go.Figure()
fig.add_scatter3d(x=embedding[:, 0], y=embedding[:, 1], z=embedding[:, 2],mode='markers', marker=marker_dict) #, color = "Euclidean distance from home", title = "Euclidean Hyp: latent space" )

fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[embedding[:, 0].min(),embedding[:, 0].max()],),
                     yaxis = dict(nticks=4, range=[embedding[:, 1].min(),embedding[:, 1].max()],),
                     zaxis = dict(nticks=4, range=[embedding[:, 2].min(),embedding[:, 2].max()],),),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10), 
    title={
        'text': "Eucl",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


fig.show()

In [ ]:
traj_act_latent = extract_umap_space_trajectory(traj, goal=0, maze_config=0, pc=placeCells, model= model3D, scaler = scaler)

In [ ]:
col = sns.color_palette("colorblind")
col

In [ ]:

marker_dict = dict(size=3,
        opacity=0.8)

fig = go.Figure()
fig.add_scatter3d(x=embedding[:, 0], y=embedding[:, 1], z=embedding[:, 2],mode='markers', marker=marker_dict) 


for g in range(3): #goal edge
    idx_g = np.where(edge == g)
    for k in range(3): #maze config
        traj_act_latent = extract_umap_space_trajectory(traj, goal=g, maze_config=k, pc=placeCells, model= model3D, scaler=scaler)
        c = (np.array(col[g+2])*256).astype(int)
        marker_dict = dict(size=6,
            opacity=1, color= [f'rgb({c[0]}, {c[1]}, {c[2]})' for i in range(traj_act_latent.shape[0])])
        fig.add_scatter3d(x=traj_act_latent[:, 0], y=traj_act_latent[:, 1], z=traj_act_latent[:, 2],mode='markers', marker=marker_dict)

"""fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[df3[0].min(),df3[0].max()],),
                     yaxis = dict(nticks=4, range=[df3[1].min(),df3[1].max()],),
                     zaxis = dict(nticks=4, range=[df3[2].min(),df3[2].max()],),),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10), 
    title={
        'text': "Eucl",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
"""

fig.show()

# Proximity Analysis


In [ ]:
def lineseg_dists(p, a, b):
    """Cartesian distance from point to line segment

    Edited to support arguments as series, from:
    https://stackoverflow.com/a/54442561/11208892

    Args:
        - p: np.array of single point, shape (2,) or 2D array, shape (x, 2)
        - a: np.array of shape (x, 2)
        - b: np.array of shape (x, 2)
    """
    # normalized tangent vectors
    d_ba = b - a
    d = np.divide(d_ba, (np.hypot(d_ba[:, 0], d_ba[:, 1])
                           .reshape(-1, 1)))

    # signed parallel distance components
    # rowwise dot products of 2D vectors
    s = np.multiply(a - p, d).sum(axis=1)
    t = np.multiply(p - b, d).sum(axis=1)

    # clamped parallel distance
    h = np.maximum.reduce([s, t, np.zeros(len(s))])

    # perpendicular distance component
    # rowwise cross products of 2D vectors  
    d_pa = p - a
    c = d_pa[:, 0] * d[:, 1] - d_pa[:, 1] * d[:, 0]

    return np.hypot(h, c)

def project(p, a, b):
    
    n = a - b
    n /= np.linalg.norm(n, 2)
    
    P = a + n*np.dot(p - a, n)
    
    return P

def graph_rel_dist(x, y, maze, cfgs_x, cfgs_y):
    #x and y are two arrays of points
    h = np.empty([maze.nb_of_trials, 2])
    c = np.empty([maze.nb_of_trials, 2])
    g0 = np.empty([maze.nb_of_trials, 2])
    g1 = np.empty([maze.nb_of_trials, 2])
    g2 = np.empty([maze.nb_of_trials, 2]) #pt/xORy/coord
    
    #extract relevant tiles for each pair of points
    for i in range(maze.nb_of_trials):
        h[i] = np.array(maze.nodeList[i]["A"])+0.5
        c[i] = np.array(maze.nodeList[i]["B"])+0.5
        g0[i] = np.array(maze.nodeList[i]["C"])+0.5
        g1[i] = np.array(maze.nodeList[i]["D"])+0.5
        g2[i] = np.array(maze.nodeList[i]["E"])+0.5

    maze_edges_v = np.array([h, g0, g1, g2])
    print(maze_edges_v.shape)
    
    #compute edge belongings for each pair of points
    d2seg_x = np.empty([x.shape[0], 4])
    d2seg_y = np.empty([y.shape[0], 4])
    
    for i in range(x.shape[0]):
        cfg_x = cfgs_x[i]
        cfg_y = cfgs_y[i]
        d2seg_x[i, :] = lineseg_dists(x[i, :], np.array([c[cfg_x], c[cfg_x], c[cfg_x], c[cfg_x]]),
                                      np.array([h[cfg_x], g0[cfg_x], g1[cfg_x], g2[cfg_x]]))
        d2seg_y[i, :] = lineseg_dists(y[i, :], np.array([c[cfg_y], c[cfg_y], c[cfg_y], c[cfg_y]]), 
                                      np.array([h[cfg_y], g0[cfg_y], g1[cfg_y], g2[cfg_y]]))
        
      
        print(str(i) + "/" + str(x.shape[0]))
    edge_bel_x = np.argmin(d2seg_x, axis = -1)
    edge_bel_y = np.argmin(d2seg_y, axis = -1)

    proj_x = np.empty([x.shape[0], 2])
    proj_y = np.empty([y.shape[0], 2])
    
    for i in range(x.shape[0]):
        cfg_x = cfgs_x[i]
        cfg_y = cfgs_y[i]
        proj_x[i, :] = project(x[i, :], maze_edges_v[edge_bel_x[i], cfg_x, :], c[cfg_x])
        proj_y[i, :] = project(y[i, :], maze_edges_v[edge_bel_y[i], cfg_y, :], c[cfg_y])
        print(str(i) + "/" + str(x.shape[0]))
    
    rel_pos_x = np.empty(x.shape[0])
    rel_pos_y = np.empty(y.shape[0])
    for i in range(x.shape[0]):
        cfg_x = cfgs_x[i]
        cfg_y = cfgs_y[i]
        rel_pos_x[i] = np.linalg.norm(c[cfg_x] - proj_x[i])/np.linalg.norm(c[cfg_x]-maze_edges_v[edge_bel_x[i], cfg_x,:])
        rel_pos_y[i] = np.linalg.norm(c[cfg_y] - proj_y[i])/np.linalg.norm(c[cfg_y]-maze_edges_v[edge_bel_y[i], cfg_y,:])
        print(str(i) + "/" + str(x.shape[0]))
    
  
    graph_rel_dist = np.empty(rel_pos_x.shape[0])

    idx_eq = np.where(edge_bel_x == edge_bel_y)[0]
    if idx_eq.size != 0:
        graph_rel_dist[idx_eq] = np.abs(rel_pos_x[idx_eq]-rel_pos_y[idx_eq])

    idx_neq = np.where(edge_bel_x != edge_bel_y)[0]
    if idx_neq.size != 0:
        graph_rel_dist[idx_neq] = np.abs(rel_pos_x[idx_neq]) + np.abs(rel_pos_y[idx_neq])


    
    return graph_rel_dist

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
SAMPLE_SIZE = 10000


corr_e1 = []
corr_e2 = []


for i in range(100):
    le = pd.DataFrame(embedding2)
    le_sample_1 = le.sample(n=SAMPLE_SIZE)
    le_sample_2 = le.sample(n=SAMPLE_SIZE)

    pos_e = pd.DataFrame(traj_2d)
    pos_e_sample_1 = pos_e.loc[le_sample_1.index]
    pos_e_sample_2 = pos_e.loc[le_sample_2.index]

   
    cfg_e = pd.DataFrame(maze_config)

   
    cfg_e_1 = cfg_e.loc[le_sample_1.index]
    cfg_e_2 = cfg_e.loc[le_sample_2.index]
    ################
    eucl_dist_pos_e = np.linalg.norm(pos_e_sample_1.to_numpy() - pos_e_sample_2.to_numpy(), axis=1)
    eucl_dist_le = np.linalg.norm(le_sample_1.to_numpy() - le_sample_2.to_numpy(), axis=1)
    
    x = pos_e_sample_1.to_numpy()
    y = pos_e_sample_2.to_numpy()
    gdist_ehyp = graph_rel_dist(x, y, maze, cfg_e_1[0].to_numpy().astype(int), cfg_e_2[0].to_numpy().astype(int))
    
   
    
    corr_e1.append(np.corrcoef(gdist_ehyp, eucl_dist_le)[1, 0])
    corr_e2.append(np.corrcoef(eucl_dist_pos_e, eucl_dist_le)[1, 0])


In [ ]:
print(np.mean(corr_e1))
print(np.std(corr_e1))

In [ ]:
print(np.mean(corr_e2))
print(np.std(corr_e2))

In [ ]:
def density_fig(x, y):

    t = np.linspace(-1, 1.2, 2000)
    
    a, b = np.polyfit(x, y, 1)

    fig = go.Figure()
    fig.add_trace(go.Histogram2dContour(
            x = x,
            y = y,
            colorscale = 'Blues',
            reversescale = False,
            xaxis = 'x',
            yaxis = 'y', 
            zmax = 475, 
            zmin=0
        ))
    
    fig.add_trace(go.Scatter(
            x = x,
            y = y,
            xaxis = 'x',
            yaxis = 'y',
            mode = 'markers',
            marker = dict(
                color = 'rgba(0, 0, 0, 0.1)',
                size = 3
            )
        ))
    fig.add_trace(go.Scatter(
            x = x,
            y = a*x+b,
            xaxis = 'x',
            yaxis = 'y',
            mode = 'markers',
            marker = dict(
                color = 'rgba(0, 0, 0, 1)',
                size = 3
            )
        ))


    fig.update_layout(
        autosize = False,
        xaxis = dict(
            zeroline = False,
            domain = [0,0.85],
            showgrid = False
        ),
        yaxis = dict(
            zeroline = False,
            domain = [0,0.85],
            showgrid = False
        ),
        xaxis2 = dict(
            zeroline = False,
            domain = [0.85,1],
            showgrid = False
        ),
        yaxis2 = dict(
            zeroline = False,
            domain = [0.85,1],
            showgrid = False
        ),
        height = 600,
        width = 600,
        bargap = 0,
  hovermode = 'closest',
        showlegend = False
    )

    return fig

In [ ]:
fig = density_fig(gdist_ehyp, eucl_dist_le)
fig.write_image("prox_g1_noise.pdf")
fig.show()
np.corrcoef(gdist_ehyp, eucl_dist_le)


In [ ]:
fig = density_fig(eucl_dist_pos_e, eucl_dist_le)
fig.write_image("prox_g2_nonoise.pdf")

fig.show()
np.corrcoef(eucl_dist_pos_e, eucl_dist_le)